# Setup

## Import modules

In [2]:
import numpy as np
import pandas as pd

In [3]:
import os
import pprint
import tempfile

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from tensorflow.keras import layers

2024-09-04 08:47:38.100262: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 08:47:38.410238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 08:47:38.410298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 08:47:38.470549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 08:47:38.593540: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-04 08:47:38.595331: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
from typing import Dict, Text

In [6]:
from datetime import datetime
import re

## Import data

Documentation for datasets: https://www.tensorflow.org/datasets/catalog/movielens

In [7]:
# Ratings data.
ratings = tfds.load("movielens/1m-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/1m-movies", split="train")

2024-09-04 08:47:47.381051: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-04 08:47:47.381536: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Transform tfds to dataframe for exploratory data analysis

In [8]:
ratings_df = tfds.as_dataframe(ratings)
movies_df = tfds.as_dataframe(movies)

In [13]:
ratings_df.info()

<class 'tensorflow_datasets.core.as_dataframe.as_dataframe.<locals>.StyledDataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   bucketized_user_age    1000209 non-null  float32
 1   movie_genres           1000209 non-null  object 
 2   movie_id               1000209 non-null  object 
 3   movie_title            1000209 non-null  object 
 4   timestamp              1000209 non-null  int64  
 5   user_gender            1000209 non-null  bool   
 6   user_id                1000209 non-null  object 
 7   user_occupation_label  1000209 non-null  int64  
 8   user_occupation_text   1000209 non-null  object 
 9   user_rating            1000209 non-null  float32
 10  user_zip_code          1000209 non-null  object 
dtypes: bool(1), float32(2), int64(2), object(6)
memory usage: 69.6+ MB


In [14]:
movies_df.info()

<class 'tensorflow_datasets.core.as_dataframe.as_dataframe.<locals>.StyledDataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_genres  3883 non-null   object
 1   movie_id      3883 non-null   object
 2   movie_title   3883 non-null   object
dtypes: object(3)
memory usage: 91.1+ KB


### Create mapping of relevant features from the tfds for the model

In [8]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_ratings": float(x["user_rating"])
})
movies = movies.map(lambda x: x["movie_title"])

# The model

## Shuffle, train, test

In [9]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(80_000)
test = ratings.skip(80_000).take(20_000)

Total Data: 1000209


## Create batches

In [10]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["user_id"])

## Get unique user_ids and movie_titles

... to later convert each user_id and movie_title to a unique integer index for the model

In [11]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [12]:
print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 3883
Unique users: 6040


## The model

In [13]:
class MovieModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("user_ratings")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

## Fitting and evaluating

In [14]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

## Shuffle, batch and cache the training and evaluation data

In [16]:
cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()



## Train the model

In [17]:
model.fit(cached_train, epochs=3)

Epoch 1/3
80/80 [==============================] - 13s 120ms/step - root_mean_squared_error: 1.2797 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0021 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0349 - factorized_top_k/top_100_categorical_accuracy: 0.0637 - loss: 6907.6526 - regularization_loss: 0.0000e+00 - total_loss: 6907.6526
Epoch 2/3
80/80 [==============================] - 9s 118ms/step - root_mean_squared_error: 1.0647 - factorized_top_k/top_1_categorical_accuracy: 2.2500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0461 - factorized_top_k/top_10_categorical_accuracy: 0.0824 - factorized_top_k/top_50_categorical_accuracy: 0.2321 - factorized_top_k/top_100_categorical_accuracy: 0.3317 - loss: 6647.9773 - regularization_loss: 0.0000e+00 - total_loss: 6647.9773
Epoch 3/3
80/80 [==============================] - 9s 118ms/step - root_mean_squared_error: 1

## Evaluate the model on the test set

In [ ]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

## Predict

In [ ]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie):
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))